In [1]:
import pandas as pd 
import numpy as np 
import os as os
from datetime import datetime
import json
import gzip
import jsonlines
import pickle

In [2]:
def writeall_jsonl_gz(filename, payload ):
    with gzip.open(filename, 'wb') as fp:
        json_writer = jsonlines.Writer(fp)
        json_writer.write_all(payload)

In [3]:
## location to save all created files 
save_folder =  "D:/Samagra/KO/Recommendations engine/IVRS transformations/"


## file name for the csv containing all the interactions for the month
#user_md_file =  'jan_feb_interaction_rank.csv'

## file name for the gzip with both ranking and interactions: 

gzip_filename_user = 'user_gzip.jsonl.gz'

gzip_filename_item = 'item_gzip.jsonl.gz'

content_md_filename = 'content_metadata.csv'

user_md_filename  =  'user_metadata_v2.csv'

In [85]:
content_md = pd.read_csv(save_folder + content_md_filename )

user_md = pd.read_csv(save_folder + user_md_filename )

C:\Users\gauta\AppData\Local\Temp\ipykernel_3008\3098426190.py:3: DtypeWarning: Columns (11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  user_md = pd.read_csv(save_folder + user_md_filename )


In [86]:
user_md['phone_number'] = user_md['phone_number'].astype(str).str[0:10]

In [192]:
user_md1= user_md[['phone_number','Activities','kharif_crops','Farmertype','district','Own_animal_large','Own_animal_small','Own_poultry']].copy()

In [194]:
user_md1['Activities'] = user_md1['Activities'].str.replace("1.0","Crop_prod").str.replace("2.0","Animal_husbandry").str.replace("3.0","Fisheries").str.replace("4.0","Forestry")

C:\Users\gauta\AppData\Local\Temp\ipykernel_3008\173023348.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  user_md1['Activities'] = user_md1['Activities'].str.replace("1.0","Crop_prod").str.replace("2.0","Animal_husbandry").str.replace("3.0","Fisheries").str.replace("4.0","Forestry")


In [195]:
for colname in user_md1.columns[1:len(user_md1.columns)]:
    user_md1.loc[:,colname] = user_md1.loc[:,colname].str.replace(",nan","").str.replace("nan,","")

### Clean up columns with multiple values by taking the max value : 

#### Changing 1,2 binary flag to 1,0

In [197]:
Binary_columns_list = user_md1.columns[5:8]
Binary_columns_list

Index(['Own_animal_large', 'Own_animal_small', 'Own_poultry'], dtype='object')

In [198]:
for colname in Binary_columns_list:
    user_md1.loc[:,colname] = user_md1.loc[:,colname].str.replace("2.0", "0.0")

C:\Users\gauta\AppData\Local\Temp\ipykernel_3008\37703357.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  user_md1.loc[:,colname] = user_md1.loc[:,colname].str.replace("2.0", "0.0")


#### Taking max value for each list :

In [199]:
take_max_cols = ['Farmertype','Own_animal_large','Own_animal_small','Own_poultry','district']

In [200]:
for colname in take_max_cols:
    user_md1.loc[user_md1[colname].notnull() ,colname] =  user_md1.loc[user_md1[colname].notnull() ,colname].str.split(",").apply(max)

#### Converting to numeric

In [201]:
convert_to_numeric_cols = ['Farmertype','Own_animal_large','Own_animal_small','Own_poultry'] 

In [202]:
for colname in convert_to_numeric_cols:
    user_md1[colname] = pd.to_numeric(user_md1[colname]).round(decimals=0).astype(object)

In [203]:
user_md1['Farmertype'].unique()

array([nan, 2.0, 6.0, 3.0, 4.0, 5.0], dtype=object)

#### Creating the list features:

In [204]:
multi_feature_cols = ['Activities','kharif_crops']

In [205]:
for colname in multi_feature_cols:
    user_md1.loc[user_md1[colname].notnull() ,colname] = user_md1[colname].str.split(",")

In [206]:
for colname  in user_md1.columns:
    user_md1[colname][pd.isna(user_md1[colname])] =  None

In [207]:
user_df = user_md1

### Creating the dictionary 

In [208]:
df_dic = user_df[['phone_number']].rename(columns= {'phone_number': 'user'})

In [209]:
df_dic['fields'] = np.empty((len(df_dic), 0)).tolist()
df_dic['id'] = df_dic['user']
df_dic['timestamp'] = 1400000000000
df_dic['event']=  'user'

In [210]:
user_dict = df_dic.to_dict('records')

In [211]:
user_df.columns

Index(['phone_number', 'Activities', 'kharif_crops', 'Farmertype', 'district',
       'Own_animal_large', 'Own_animal_small', 'Own_poultry'],
      dtype='object')

### Collating all the fields into the dictionary:  

In [212]:
user_df.columns[1:user_df.shape[1]]

Index(['Activities', 'kharif_crops', 'Farmertype', 'district',
       'Own_animal_large', 'Own_animal_small', 'Own_poultry'],
      dtype='object')

In [213]:
for i in range(0, user_df.shape[0]):
    for colname in  user_df.columns[1:user_df.shape[1]]:
        if (user_df.loc[i,colname] is not None):
            dict_fields = {}
            dict_fields.update({ "name": colname,
                            "value": user_df.loc[i,colname]
            })
            user_dict[i]['fields'].append(dict_fields)

In [214]:
user_dict[4]['fields']

[{'name': 'Activities', 'value': ['Crop_prod']},
 {'name': 'kharif_crops', 'value': ['paddy', 'greengram']},
 {'name': 'Farmertype', 'value': 2.0},
 {'name': 'district', 'value': 'KALAHANDI'}]

### Checking item metadata: 

In [163]:
content_md1 = content_md.copy()

In [165]:
content_md1['month'] = content_md1['month'].round(decimals=0).astype(object)
content_md1['duration'] = content_md1['duration'].round(decimals=0).astype(object)

In [166]:
content_md1.dtypes

pushcall_id    object
base_tag       object
tag_2          object
tag_3          object
crop           object
month          object
duration       object
dtype: object

In [167]:
multi_feature_cols = ['crop']
for colname in multi_feature_cols:
    content_md1.loc[content_md1[colname].notnull() ,colname] = content_md1[colname].str.split(",")

In [169]:
for colname  in content_md1.columns:
    content_md1[colname][pd.isna(content_md1[colname])] =  None

####  Creating the dictionary: 

In [170]:
df_dic = content_md1[['pushcall_id']].rename(columns= {'pushcall_id': 'item'})

In [171]:
df_dic['fields'] = np.empty((len(df_dic), 0)).tolist()
df_dic['id'] = df_dic['item']
df_dic['timestamp'] = 1400000000000
df_dic['event']=  'item'

In [172]:
item_dict = df_dic.to_dict('records')

In [174]:
item_dict[0:4]

[{'item': '20210703_cucurbits_downy_mildew_dpm_crop',
  'fields': [],
  'id': '20210703_cucurbits_downy_mildew_dpm_crop',
  'timestamp': 1400000000000,
  'event': 'item'},
 {'item': '20210703_maize_faw_dpm_crop',
  'fields': [],
  'id': '20210703_maize_faw_dpm_crop',
  'timestamp': 1400000000000,
  'event': 'item'},
 {'item': '20210703_sugarcane_top_shoot_borer_dpm_crop',
  'fields': [],
  'id': '20210703_sugarcane_top_shoot_borer_dpm_crop',
  'timestamp': 1400000000000,
  'event': 'item'},
 {'item': '20210801_brinjal_shoot_and_fruit_borer_dpm_crop',
  'fields': [],
  'id': '20210801_brinjal_shoot_and_fruit_borer_dpm_crop',
  'timestamp': 1400000000000,
  'event': 'item'}]

In [176]:
for i in range(0, content_md1.shape[0]):
    for colname in  content_md1.columns[1:content_md1.shape[1]]:
        if  content_md1.loc[i,colname] is not None:
            dict_fields = {}

            dict_fields.update({ "name": colname,
                            "value": content_md1.loc[i,colname]
            })
            item_dict[i]['fields'].append(dict_fields)

In [178]:
payload  = item_dict
writeall_jsonl_gz(save_folder + gzip_filename_item, payload )

In [215]:
payload  = user_dict
writeall_jsonl_gz(save_folder + gzip_filename_user, payload )

### WIP

In [398]:

# with open('save_folder' + 'items_dic_Jan_Feb.pickle', 'wb') as handle:
#     pickle.dump(item_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [399]:

# with open('save_folder' + 'user_dict_Jan_Feb.pickle', 'wb') as handle:
#     pickle.dump(user_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [400]:
# events_list =  []
# events_list.extend(user_dict)
# events_list.extend(item_dict)

# ##saving the created events: 

# payload  = events_list
# writeall_jsonl_gz(save_folder + gzip_filename, payload )

In [54]:
#rank_interactions_dic = [json.loads(line) for line in open('ranking_and_interaction_Jan_Feb.jsonl/ranking_and_interaction_Jan_Feb.jsonl', 'r',encoding="utf8")]


In [ ]:
# events_list =  []
# events_list.extend(user_dict)
# events_list.extend(item_dict)
# events_list.extend(rank_interactions_dic)

In [ ]:
# payload  = events_list
# writeall_jsonl_gz(save_folder + 'all_events.jsonl.gzip', payload )